## Исследование надёжности заёмщиков

### Шаг 1. Импорт и загрузка данных

In [2]:
# импортируем данные и сохраним таблицу в переменной data_client
import pandas as pd
data_client = pd.read_csv('/datasets/data.csv')
#data_client.head(10)

In [3]:
# просмотрим информацию о таблице
data_client.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

В таблице представлены данные о 21 525 заемщиках разного возраста c разным количеством детей, трудовым стажем, уровнем образования, семейным положением, типом занятости, кредитной историей и целями привлечения займа. В таблице есть пропущенные значения в столбцах трудовой стаж в днях и ежемесячный доход. Количество этих значений в указанных столбцах отличаются на одну и ту же величину. Возможно, значения пропущены в одних и тех же строках. К причинам подобных пропусков я бы отнесла неофициальное трудоустройство с неофициальной заработной платой; фриланс.


### Шаг 2. Предобработка данных
### Обработка пропусков

In [4]:
# посчитаем количество пропусков в столбцах
data_client.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

### Вывод
Количество значений в столбцах трудовой стаж в днях и суммарный доход заемщика отличаются от общего количества клиентов на одно и то же значение. Тип данных у этих столбцов - дробные числа. В столбце с трудовым стажем есть отрицательные значения. 

In [5]:
# заменим отрицательные значения на положительные
data_client['days_employed'] = data_client['days_employed'].abs()
data_client.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [6]:
# удалим строки с пропущенными данными, чтобы проверить одни ли и те же строки с пропущенными значеними в графах со стажем
# и доходом. А затем закомментируем удаление, чтобы продолжить обработку таблицы и заполнить пропуски 
#data_client.dropna(inplace=True)
data_client.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Количество строк стало 19351, то есть удалились только строки с информацией о заемщиках без дохода и без опыта работы и это одни и те же люди. Заполним пропуски.

---
## Комментарий от наставника

Мы проверяем влияние не на выдачу кредита, а на погашение. Т.е. всем клиентам в данных кредиты были одобрены. Вряд ли это возможно с нулевым уровнем дохода. Также удалено около 10% данных, что довольно много. Попробуй сохранить данные и заполнить пропуски. Например, используй заполнение пропусков в соответствии с типом дохода. Тогда пенсионеры будут получать также, как и другие пенсионеры и т.д. Все-таки в данных есть достаточное число групп, заполнять пропуски в стаже, скажем, у студентов и пенсионеров одним и тем же значением не очень то правильно. То же самое касается и столбца с доходами: пропуски у предпринимателя и пенсионера заполняются одним числом, что вызывает вопросы. Подумай, как лучше обработать пропуски. +

---


In [7]:
# Для заполнения пропусков воспользуемся столбцом о типе занятости income_type:
data_client['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [8]:
# Заполним пропущенные значения столбца о стаже средним стажем по каждому типу занятости.
# Для этого посчитаем средний стаж по типам занятости:
sotr_mean_days_employed = data_client[data_client['income_type'] == 'сотрудник']['days_employed'].mean()
comp_mean_days_employed = data_client[data_client['income_type'] == 'компаньон']['days_employed'].mean()
pens_mean_days_employed = data_client[data_client['income_type'] == 'пенсионер']['days_employed'].mean()
gos_mean_days_employed = data_client[data_client['income_type'] == 'госслужащий']['days_employed'].mean()
pp_mean_days_employed = data_client[data_client['income_type'] == 'предприниматель ']['days_employed'].mean()
bezr_mean_days_employed = data_client[data_client['income_type'] == 'безработный']['days_employed'].mean()
stud_mean_days_employed = data_client[data_client['income_type'] == 'студент']['days_employed'].mean()
dekr_mean_days_employed = data_client[data_client['income_type'] == 'в декрете']['days_employed'].mean()


Средний стаж сотрудников составляет 2326.50
Средний стаж компаньонов составляет 2111.52
Средний стаж пенсионеров составляет 365003.49
Средний стаж госслужащих составляет 3399.90
Средний стаж предпринимателей составляет nan
Средний стаж безработных составляет 366413.65
Средний стаж студентов составляет 578.75
Средний стаж клиентов в декрете составляет 3296.76


In [9]:
# Заполним пропущенные значения столбца о стаже по типу "сотрудник"
data_client.loc[data_client['income_type'] == 'сотрудник', 'days_employed']=data_client.loc[data_client['income_type'] == 'сотрудник', 'days_employed'].fillna(value=sotr_mean_days_employed)

In [10]:
# Заполним пропущенные значения столбца о стаже по типу "компаньон"
data_client.loc[data_client['income_type'] == 'компаньон', 'days_employed']=data_client.loc[data_client['income_type'] == 'компаньон', 'days_employed'].fillna(value=comp_mean_days_employed)

In [11]:
# Заполним пропущенные значения столбца о стаже по типу "пенсионер"
data_client.loc[data_client['income_type'] == 'пенсионер', 'days_employed']=data_client.loc[data_client['income_type'] == 'пенсионер', 'days_employed'].fillna(value=pens_mean_days_employed)

In [12]:
# Заполним пропущенные значения столбца о стаже по типу "госслужащий"
data_client.loc[data_client['income_type'] == 'госслужащий', 'days_employed']=data_client.loc[data_client['income_type'] == 'госслужащий', 'days_employed'].fillna(value=gos_mean_days_employed)

In [13]:
# Средний стаж предпринимателей не посчитался, возможно по ним отсутствуют данные. После заполнения всех пропусков,
# которые возможно заполнить, удалим эти строки. Их всего 2. 
# Средний стаж безработных слишком большой, тоже удалим строки с безработными:

data_client = data_client[data_client['income_type'] != 'безработный']
data_client['income_type'].value_counts()

# В типах занятости "студент" и "в декрете" по одному клиенту, заполнять пропуски нет необходимости - их нет.

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [14]:
# рассмотрим столбец days_employed
data_client.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21523 entries, 0 to 21524
Data columns (total 12 columns):
children            21523 non-null int64
days_employed       21522 non-null float64
dob_years           21523 non-null int64
education           21523 non-null object
education_id        21523 non-null int64
family_status       21523 non-null object
family_status_id    21523 non-null int64
gender              21523 non-null object
income_type         21523 non-null object
debt                21523 non-null int64
total_income        19349 non-null float64
purpose             21523 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


### Вывод

В столбце о стаже заемщиков отсутствующие значения были заполнены средними по типу занятости каждого клиента. Строки с безработными заемщиками удалены по причине слишком большого стажа, таких строк всего 2, и критично они повлиять на результаты оценки различных факторов на погашение кредита не могут. Также в этом столбце содержатся не всегда адекватные значения стажа работы в днях, но информация о стаже не требуется для решения поставленных в проекте задач.

In [15]:
# рассмотрим столбец 'children':

data_client['children'].value_counts()

 0     14148
 1      4817
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

### Вывод

В столбце с количеством детей есть информация о 67 клиентах с 20 детьми и о 44 с количеством детей равном отрицательному числу. 

In [16]:
# Рассмотрим строки с клиентами количество детей которых равно 20:

children_quantity = ['20']
data_client[data_client.children.isin(children_quantity)].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
720,20,855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости
1074,20,3310.411598,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования
2510,20,2714.161249,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью
2941,20,2161.591519,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля
3302,20,3399.896902,35,среднее,1,Не женат / не замужем,4,F,госслужащий,0,NaN,профильное образование
3396,20,2111.524398,56,высшее,0,женат / замужем,0,F,компаньон,0,NaN,высшее образование
3671,20,913.161503,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,101255.492076,на покупку подержанного автомобиля
3697,20,2907.910616,40,среднее,1,гражданский брак,1,M,сотрудник,0,115380.694664,на покупку подержанного автомобиля
3735,20,805.044438,26,высшее,0,Не женат / не замужем,4,M,сотрудник,0,137200.646181,ремонт жилью


### Вывод

In [17]:
# заменим количество детей с 20 на 2:
data_client['children'] = data_client['children'].replace(20, 2)
data_client['children'].value_counts()

 0    14148
 1     4817
 2     2131
 3      330
-1       47
 4       41
 5        9
Name: children, dtype: int64

In [18]:
# Рассмотрим строки с клиентами количество детей которых равно -1:

children_quantity = ['-1']
data_client[data_client.children.isin(children_quantity)].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
291,-1,4417.703588,46,среднее,1,гражданский брак,1,F,сотрудник,0,102816.346412,профильное образование
705,-1,902.084528,50,среднее,1,женат / замужем,0,F,госслужащий,0,137882.899271,приобретение автомобиля
742,-1,3174.456205,57,среднее,1,женат / замужем,0,F,сотрудник,0,64268.044444,дополнительное образование
800,-1,349987.852217,54,среднее,1,Не женат / не замужем,4,F,пенсионер,0,86293.724153,дополнительное образование
941,-1,365003.491245,57,Среднее,1,женат / замужем,0,F,пенсионер,0,NaN,на покупку своего автомобиля
1363,-1,1195.264956,55,СРЕДНЕЕ,1,женат / замужем,0,F,компаньон,0,69550.699692,профильное образование
1929,-1,1461.303336,38,среднее,1,Не женат / не замужем,4,M,сотрудник,0,109121.569013,покупка жилья
2073,-1,2539.761232,42,среднее,1,в разводе,3,F,компаньон,0,162638.609373,покупка жилья
3814,-1,3045.290443,26,Среднее,1,гражданский брак,1,F,госслужащий,0,131892.785435,на проведение свадьбы
4201,-1,901.101738,41,среднее,1,женат / замужем,0,F,госслужащий,0,226375.766751,операции со своей недвижимостью


In [19]:
# заменим количество детей с -1 на 1
data_client['children'] = data_client['children'].replace(-1, 1)

# проверим результат
data_client['children'].value_counts()

0    14148
1     4864
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

### Вывод

In [20]:
# рассмотрим столбец dob_years
data_client['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    559
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    496
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

### Вывод

In [21]:
# заменим значение возраста заемщика 0 на медиану значений возраста заемщиков:
data_client['dob_years'] = data_client['dob_years'].replace(0, data_client['dob_years'].median())
data_client['dob_years'].value_counts()

42    698
35    617
40    609
41    607
34    603
38    598
33    581
39    573
31    559
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    496
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

### Вывод

In [22]:
# рассмотрим столбец education
data_client['education'].value_counts()

среднее                13749
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   267
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

### Вывод

In [23]:
# рассмотрим столбец family_status
data_client['family_status'].value_counts()

женат / замужем          12379
гражданский брак          4176
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

### Вывод

In [24]:
# рассмотрим столбец gender
data_client['gender'].value_counts()

F      14235
M       7287
XNA        1
Name: gender, dtype: int64

### Вывод

In [25]:
data_client = data_client[data_client['gender'] != 'XNA']
data_client['gender'].value_counts()

F    14235
M     7287
Name: gender, dtype: int64

In [26]:
# рассмотрим столбец income_type
data_client['income_type'].value_counts()

сотрудник          11119
компаньон           5084
пенсионер           3856
госслужащий         1459
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

### Вывод

In [27]:
# рассмотрим столбец total_income 

# data_client.info()

# в этом столбце 19 348 пропущенных значений, заполним их по аналогии со столбцом со стажем работы: посчитаем среднее значение
# ежемесячного дохода по типам занятости клиентов:
sotr_mean_total_income = data_client[data_client['income_type'] == 'сотрудник']['total_income'].mean()
comp_mean_total_income = data_client[data_client['income_type'] == 'компаньон']['total_income'].mean()
pens_mean_total_income = data_client[data_client['income_type'] == 'пенсионер']['total_income'].mean()
gos_mean_total_income = data_client[data_client['income_type'] == 'госслужащий']['total_income'].mean()
pp_mean_total_income = data_client[data_client['income_type'] == 'предприниматель ']['total_income'].mean()
stud_mean_total_income = data_client[data_client['income_type'] == 'студент']['total_income'].mean()
dekr_mean_total_income = data_client[data_client['income_type'] == 'в декрете']['total_income'].mean()

Средний ежемесячный доход сотрудников составляет 161380.26
Средний ежемесячный доход компаньонов составляет 202417.14
Средний ежемесячный доход пенсионеров составляет 137127.47
Средний ежемесячный доход госслужащих составляет 170898.31
Средний ежемесячный доход предпринимателей составляет nan
Средний ежемесячный доход студентов составляет 98201.63
Средний ежемесячный доход клиентов в декрете составляет 53829.13


In [28]:
# Заполним пропущенные значения столбца о ежемесячном доходе по типу "сотрудник"
data_client.loc[data_client['income_type'] == 'сотрудник', 'total_income']=data_client.loc[data_client['income_type'] == 'сотрудник', 'total_income'].fillna(value=sotr_mean_total_income)

# по типу "компаньон":
data_client.loc[data_client['income_type'] == 'компаньон', 'total_income']=data_client.loc[data_client['income_type'] == 'компаньон', 'total_income'].fillna(value=comp_mean_total_income)

# по типу "пенсионер":
data_client.loc[data_client['income_type'] == 'пенсионер', 'total_income']=data_client.loc[data_client['income_type'] == 'пенсионер', 'total_income'].fillna(value=pens_mean_total_income)

# по типу "госслужащий":
data_client.loc[data_client['income_type'] == 'госслужащий', 'total_income']=data_client.loc[data_client['income_type'] == 'госслужащий', 'total_income'].fillna(value=gos_mean_total_income)

# Средний стажежемесячный доход предпринимателей не посчитался, возможно по ним отсутствуют данные. После заполнения всех 
# пропусков, которые возможно заполнить, удалим эти строки. Их всего 2. 

# В типах занятости "студент" и "в декрете" по одному клиенту, заполнять пропуски нет необходимости - их нет.

In [29]:
# проверим заполнение:

data_client.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21522 entries, 0 to 21524
Data columns (total 12 columns):
children            21522 non-null int64
days_employed       21521 non-null float64
dob_years           21522 non-null int64
education           21522 non-null object
education_id        21522 non-null int64
family_status       21522 non-null object
family_status_id    21522 non-null int64
gender              21522 non-null object
income_type         21522 non-null object
debt                21522 non-null int64
total_income        21521 non-null float64
purpose             21522 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


In [30]:
# Проверим нет ли отрицательных значений: сравним сумму значений исходного столбца с суммой значений столбца, взятых по модулю,
# если они равны, то отрицательных значений нет

if data_client['total_income'].sum() == data_client['total_income'].abs().sum():
    print('Отрицательные значения в столбце с ежемесячным доходом отсутствуют')

Отрицательные значения в столбце с ежемесячным доходом отсутствуют


In [31]:
# удалим 1 пропущенную строку
data_client.dropna(inplace=True)
data_client.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21521 entries, 0 to 21524
Data columns (total 12 columns):
children            21521 non-null int64
days_employed       21521 non-null float64
dob_years           21521 non-null int64
education           21521 non-null object
education_id        21521 non-null int64
family_status       21521 non-null object
family_status_id    21521 non-null int64
gender              21521 non-null object
income_type         21521 non-null object
debt                21521 non-null int64
total_income        21521 non-null float64
purpose             21521 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


### Вывод

In [32]:
# рассмотрим столбец purpose
data_client['purpose'].value_counts()


свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   652
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      623
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              611
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

### Вывод

### Замена типа данных

In [33]:
# заменим столбы с типом данных float64 на int64 и проверим как произошла замена:
data_client['days_employed'] = data_client['days_employed'].astype('int')
data_client['total_income'] = data_client['total_income'].astype('int')
data_client.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21521 entries, 0 to 21524
Data columns (total 12 columns):
children            21521 non-null int64
days_employed       21521 non-null int64
dob_years           21521 non-null int64
education           21521 non-null object
education_id        21521 non-null int64
family_status       21521 non-null object
family_status_id    21521 non-null int64
gender              21521 non-null object
income_type         21521 non-null object
debt                21521 non-null int64
total_income        21521 non-null int64
purpose             21521 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


---
## Комментарий от наставника

Исходя из общей информации о таблице мы видим, что два столбца имеют вещественный тип данных, который и надо заменить на целочисленный. Это замена произведена верно. Также хорошо бы освоить метод to_numeric(), он пригодится в дальнейшем. 

---

---
## Комментарий от студента
В таблице представлены строковые данные в столбцах о типе образования, семейном положении, поле, типе дохода и цели привлечения займа. Нет возможности и необходимости переводить эти данные в количественный тип данных.

---

In [34]:
# рассмотрим значения столбца с уровнем образования заемщиков
data_client['education'].unique()


array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Много повторяющихся значений, буквы разного регистра. Приведем все значения к нижнему регистру:

In [35]:
data_client['education'] = data_client['education'].str.lower()
# и проверим на наличие дубликатов
data_client['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

### Вывод

Повторяющиеся значения исключены приведением всех значений столбца к нижнему регистру.

### Обработка дубликатов

In [36]:
data_client.duplicated().sum()

72

In [37]:
# удалим дубликаты 
data_client = data_client.drop_duplicates().reset_index()

In [38]:
# и проверим еще раз на наличие дубликатов
data_client.duplicated().sum()

0

### Лемматизация

In [39]:
from pymystem3 import Mystem
m = Mystem()
# лемматизируем столбец с целью привлечения займа клиентом
# для этого определим функцию lemma_in_row(purpose)

def lemma_in_row(purpose):
    lemma = ' '.join(m.lemmatize(purpose))
    return lemma

# создадим столбец purpose_lemma, в котором будут леммы столбца purpose
data_client['purpose_lemma'] = data_client['purpose'].apply(lemma_in_row)

# посчитаем количество уникальных лемм и выведем их в порядке убывания:
from collections import Counter
purpose_lemma_count = Counter(data_client['purpose_lemma'])
purpose_lemma_count.most_common()

[('автомобиль \n', 972),
 ('свадьба \n', 791),
 ('на   проведение   свадьба \n', 767),
 ('сыграть   свадьба \n', 765),
 ('операция   с   недвижимость \n', 675),
 ('покупка   коммерческий   недвижимость \n', 661),
 ('операция   с   жилье \n', 652),
 ('операция   с   коммерческий   недвижимость \n', 650),
 ('покупка   жилье   для   сдача \n', 650),
 ('покупка   жилье \n', 646),
 ('жилье \n', 646),
 ('покупка   жилье   для   семья \n', 638),
 ('строительство   собственный   недвижимость \n', 635),
 ('недвижимость \n', 633),
 ('операция   со   свой   недвижимость \n', 627),
 ('строительство   жилой   недвижимость \n', 624),
 ('покупка   недвижимость \n', 620),
 ('покупка   свой   жилье \n', 620),
 ('строительство   недвижимость \n', 619),
 ('ремонт   жилье \n', 606),
 ('покупка   жилой   недвижимость \n', 605),
 ('на   покупка   свой   автомобиль \n', 505),
 ('заниматься   высокий   образование \n', 496),
 ('сделка   с   подержанный   автомобиль \n', 486),
 ('на   покупка   подержать   авт

### Вывод

---
## Комментарий от наставника

Все верно, метод lemmatize() использован правильно. Хорошо бы на данном этапе подробно объяснить выбор лемм +.

---

### Категоризация данных

In [40]:
# создадим столбец purpose_category на основании данных лемматизации столбца purpose_lemma
def purpose_category(purpose_lemma):
    if 'автомобиль' in purpose_lemma:
        return "сделка с автомобилем"
    if "образование" in purpose_lemma:
        return "образование"
    if "свадьба" in purpose_lemma:
        return "свадьба"
    if "недвижимость" in purpose_lemma:
        return "сделка с недвижимостью"
    if "строительство" in purpose_lemma:
        return "сделка с недвижимостью"
    if "жилье" in purpose_lemma:
        return "сделка с недвижимостью"
data_client['purpose_category'] = data_client['purpose_lemma'].apply(purpose_category)

# Выведем статистику по целям привлечения займа
data_client['purpose_category'].value_counts()

сделка с недвижимостью    10807
сделка с автомобилем       4306
образование                4013
свадьба                    2323
Name: purpose_category, dtype: int64

In [41]:
# удалим столбец purpose
data_client.drop('purpose', axis=1, inplace=True) 

# удалим столбец purpose_lemma
data_client.drop('purpose_lemma', axis=1, inplace=True)

# посмотрим на результат
data_client

,index,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose_category
0,0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,сделка с недвижимостью
1,1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,сделка с автомобилем
2,2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,сделка с недвижимостью
3,3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21444,21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,сделка с недвижимостью
21445,21521,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21446,21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,сделка с недвижимостью
21447,21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,сделка с автомобилем


### Вывод

После проведения категоризации данных в столбце purpose цели привлечения займа разделены на 4 группы: сделка с недвижимостью, сделка с автомобилем, образование, свадьба.

In [42]:
# для удобства поиска ответа на поставленную задачу о зависимости между уровнем дохода и возвратом кредита в срок сгруппируем
# доходы заемщиков в группы
# для этого проверим максимальное, минимальное, среднее и медианное значение доходов заемщиков:

print('Минимальный доход заемщиков составляет', data_client['total_income'].min(), 'у.е.')
print('Медиана доходов заемщиков составляет {:.0f}'.format(data_client['total_income'].median()), 'у.е.')
print('Средний доход заемщиков составляет {:.0f}'.format(data_client['total_income'].mean()), 'у.е.') 
print('Максимальный доход заемщиков составляет', data_client['total_income'].max(), 'у.е.')

data_client['total_income'].describe()

Минимальный доход заемщиков составляет 20667 у.е.
Медиана доходов заемщиков составляет 151846 у.е.
Средний доход заемщиков составляет 167418 у.е.
Максимальный доход заемщиков составляет 2265604 у.е.


count    2.144900e+04
mean     1.674181e+05
std      9.804250e+04
min      2.066700e+04
25%      1.076200e+05
50%      1.518460e+05
75%      2.024170e+05
max      2.265604e+06
Name: total_income, dtype: float64

---
## Комментарий от наставника

Эти и другие параметры можно получить при использовании метода describe(). 

---

In [43]:
# сгруппируем доходы заемщиков 

def group_of_income(row):
    
    income = row['total_income']
  
 # возьмем доходы с отклонением примерно в 40 000 у.е. от медианы как Средний тип доходов, все что ниже - Доходы ниже среднего
 # выше - Доходы выше среднего, а очень большие доходы выделим в тип - Высокие доходы
    if income <= 120000:
        return 'Доходы ниже среднего'
   
    if income <= 190000 and income > 110000:
        return 'Средние доходы'
    
    if income <= 290000 and income > 190000:
        return 'Доходы выше среднего'
    
    if income <= 2265604 and income > 290000:
        return 'Высокие доходы'

data_client['group_in'] = data_client.apply(group_of_income, axis=1)
data_client
data_client['group_in'].value_counts()

Средние доходы          8356
Доходы ниже среднего    6844
Доходы выше среднего    4594
Высокие доходы          1655
Name: group_in, dtype: int64

### Шаг 3. Определение зависимостей

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [44]:
# посчитаем процент заемщиков, которые имели задолженность по возврату кредита:

delay_per = data_client['debt'][(data_client['debt'] == 1)].count() / data_client['debt'][(data_client['debt'] == 0)].count()
print('В {:.2%}'.format(delay_per), 'случаев кредит не возвращался вовремя')

В 8.83% случаев кредит не возвращался вовремя


In [45]:
# для оценки влияния количества детей заемщика на факт погашения кредита в срок
# построим сводную таблицу на основании обработанной исходной таблицы
# группировку проведем по столбцу children, столбец, по значениям которого проведем группировку - debt
# значения, по которым будет сводная таблица  - значения столбца family_status, для которого применим функцию count():

data_client_pivot_children = data_client.pivot_table(index=['children'], columns='debt', values = 'family_status', aggfunc='count')

# для удобства восприятия переименуем в полученной таблице названия столбцов с 0 и 1 на Добросовестного плательщика и Должник
# соответственно:

new_names = ['Добросовестный плательщик', 'Должник']
data_client_pivot_children.set_axis(new_names, axis = "columns", inplace = True)


# создадим в полученной таблице столбец, в котором будет рассчитана доля должников с разным количеством детей и назовем его
# Доля должников, и столбец, в котором будет рассчитано отклонение доли должников с разным количеством детей от общего уровня
# должников и назовем его Отклонение от общего уровня должников

data_client_pivot_children['Доля должников'] = data_client_pivot_children['Должник'] / data_client_pivot_children['Добросовестный плательщик']
data_client_pivot_children['Отклонение от общего уровня должников'] = data_client_pivot_children['Доля должников'] - delay_per
data_client_pivot_children

,Добросовестный плательщик,Должник,Доля должников,Отклонение от общего уровня должников
children,,,,
0,13024.0,1063.0,0.081619,-0.006666
1,4410.0,444.0,0.100680,0.012396
2,1926.0,202.0,0.104881,0.016596
3,303.0,27.0,0.089109,0.000824
4,37.0,4.0,0.108108,0.019824
5,9.0,NaN,NaN,NaN


### Вывод

На основании сводной таблицы можно сделать вывод, что вероятность появления задолженности по возврату кредита увеличивается c появлением в семье детей. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [46]:
# для оценки влияния семейного положения заемщика на факт погашения кредита в срок
# построим сводную таблицу на основании обработанной исходной таблицы
# группировку проведем по столбцу family_status, столбец, по значениям которого проведем группировку - debt
# значения, по которым будет сводная таблица  - значения столбца family_status_id, для которого применим функцию count():

data_client_pivot_family_status = data_client.pivot_table(index=['family_status'], columns='debt', values = 'family_status_id', aggfunc='count')


# для удобства восприятия переименуем в полученной таблице названия столбцов с 0 и 1 на Добросовестного плательщика и Должник
# соответственно:

new_names = ['Добросовестный плательщик', 'Должник']
data_client_pivot_family_status.set_axis(new_names, axis = "columns", inplace = True)


# создадим в полученной таблице столбец, в котором будет рассчитана доля должников с разным семейным положением и назовем его 
# Доля должников, и столбец, в котором будет рассчитано отклонение доли должников с разным семейным положением от общего уровня
# должников и назовем его Отклонение от общего уровня должников

data_client_pivot_family_status['Доля должников'] = data_client_pivot_family_status['Должник'] / data_client_pivot_family_status['Добросовестный плательщик']
data_client_pivot_family_status['Отклонение от общего уровня должников'] = data_client_pivot_family_status['Доля должников'] - delay_per
data_client_pivot_family_status

,Добросовестный плательщик,Должник,Доля должников,Отклонение от общего уровня должников
family_status,,,,
Не женат / не замужем,2536,274,0.108044,0.019760
в разводе,1110,85,0.076577,-0.011708
вдовец / вдова,896,63,0.070312,-0.017972
гражданский брак,3760,388,0.103191,0.014907
женат / замужем,11407,930,0.081529,-0.006756


### Вывод

На основании сводной таблицы можно сделать вывод, что заемщики, живущие в гражданском браке, и неженатые/незамужние заемщики чаще, чем остальные, имеют задолженность по оплате кредита, а самые ответственные заемщики - вдовцы.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [47]:
# для оценки влияния уровня дохода заемщика на факт погашения кредита в срок
# построим сводную таблицу на основании обработанной исходной таблицы
# группировку проведем по столбцу group_in, столбец, по значениям которого проведем группировку - debt
# значения, по которым будет сводная таблица  - значения столбца family_status_id, для которого применим функцию count():

data_client_pivot_groupf_in = data_client.pivot_table(index=['group_in'], columns='debt', values='family_status_id', aggfunc='count')

# для удобства восприятия переименуем в полученной таблице названия столбцов с 0 и 1 на Добросовестного плательщика и Должник
# соответственно:

new_names = ['Добросовестный плательщик', 'Должник']
data_client_pivot_groupf_in.set_axis(new_names, axis = "columns", inplace = True)


# создадим в полученной таблице столбец, в котором будет рассчитана доля должников с разным уровнем дохода и назовем его 
# Доля должников, и столбец, в котором будет рассчитано отклонение доли должников с разным уровнем дохода от общего уровня
# должников и назовем его Отклонение от общего уровня должников

data_client_pivot_groupf_in['Доля должников'] = data_client_pivot_groupf_in['Должник'] / data_client_pivot_groupf_in['Добросовестный плательщик']
data_client_pivot_groupf_in['Отклонение от общего уровня должников'] = data_client_pivot_groupf_in['Доля должников'] - delay_per
data_client_pivot_groupf_in

,Добросовестный плательщик,Должник,Доля должников,Отклонение от общего уровня должников
group_in,,,,
Высокие доходы,1535,120,0.078176,-0.010109
Доходы выше среднего,4266,328,0.076887,-0.011398
Доходы ниже среднего,6294,550,0.087385,-0.000900
Средние доходы,7614,742,0.097452,0.009168


### Вывод

На основании сводной таблицы можно сделать вывод, что заемщики с доходами вышего среднего  - самые ответственные заемщики, которые в меньшем количестве случаев, чем остальные, имели задолженность по оплате кредита. А заемщики со средними доходами имеют большую склонность к задержке оплаты кредита.

- Как разные цели кредита влияют на его возврат в срок?

In [48]:
# для оценки влияния целей кредита на факт погашения кредита в срок
# построим сводную таблицу на основании обработанной исходной таблицы
# группировку проведем по столбцу purpose_category, столбец, по значениям которого проведем группировку - debt
# значения, по которым будет сводная таблица  - значения столбца family_status_id, для которого применим функцию count():

data_client_pivot_purpose_category = data_client.pivot_table(index=['purpose_category'], columns='debt', values='family_status_id', aggfunc='count')

# для удобства восприятия переименуем в полученной таблице названия столбцов с 0 и 1 на Добросовестного плательщика и Должник
# соответственно:

new_names = ['Добросовестный плательщик', 'Должник']
data_client_pivot_purpose_category.set_axis(new_names, axis = "columns", inplace = True)


# создадим в полученной таблице столбец, в котором будет рассчитана доля должников с разным целями кредита и назовем его 
# Доля должников, и столбец, в котором будет рассчитано отклонение доли должников с разными целями кредита от общего уровня
# должников и назовем его Отклонение от общего уровня должников

data_client_pivot_purpose_category['Доля должников'] = data_client_pivot_purpose_category['Должник'] / data_client_pivot_purpose_category['Добросовестный плательщик']
data_client_pivot_purpose_category['Отклонение от общего уровня должников'] = data_client_pivot_purpose_category['Доля должников'] - delay_per
data_client_pivot_purpose_category

,Добросовестный плательщик,Должник,Доля должников,Отклонение от общего уровня должников
purpose_category,,,,
образование,3643,370,0.101565,0.013280
свадьба,2137,186,0.087038,-0.001247
сделка с автомобилем,3903,403,0.103254,0.014969
сделка с недвижимостью,10026,781,0.077897,-0.010387


### Вывод

На основании сводной таблицы можно сделать вывод, что заемщики, взявшие кредит на сделки с недвижимостью, более добросовестно выплачивают его. В свою очередь, кредитование на сделки с автомобилем имеют более высокие риски по выплате в срок.

### Шаг 4. Общий вывод

С исходной таблицей проведена предобработка данных - исключены и заполнены пропуски, удалены дубликаты, проведена необходимая категоризация. 
Проведены расчеты по оценке влияния появления детей, семейного положения, ежемесячного дохода и целей привлечения займа на факт погашения займа в срок.


In [49]:
print('В {:.2%}'.format(delay_per), 'случаев кредит не возвращался вовремя')

В 8.83% случаев кредит не возвращался вовремя


In [50]:
data_client_pivot_children

,Добросовестный плательщик,Должник,Доля должников,Отклонение от общего уровня должников
children,,,,
0,13024.0,1063.0,0.081619,-0.006666
1,4410.0,444.0,0.100680,0.012396
2,1926.0,202.0,0.104881,0.016596
3,303.0,27.0,0.089109,0.000824
4,37.0,4.0,0.108108,0.019824
5,9.0,NaN,NaN,NaN


Заемщики, не имеющие детей более надежны в отношении погашения платежа по кредиту в срок почти на 1% по сравнению с общим количеством должников. С появлением первого и второго ребенка надежность снижается. А в многодетных семьях надежность заемщиков уменьшается на 2% по отношению к среднему уровню надежности.


In [51]:
data_client_pivot_family_status

,Добросовестный плательщик,Должник,Доля должников,Отклонение от общего уровня должников
family_status,,,,
Не женат / не замужем,2536,274,0.108044,0.019760
в разводе,1110,85,0.076577,-0.011708
вдовец / вдова,896,63,0.070312,-0.017972
гражданский брак,3760,388,0.103191,0.014907
женат / замужем,11407,930,0.081529,-0.006756


Наиболее надежными плательщиками кредитов являются вдовцы и вдовы, среди них должников почти на 2% меньше, чем в среднем, а незамужние и неженатые должники на 2% превышают средний процент должников.


In [52]:
data_client_pivot_groupf_in

,Добросовестный плательщик,Должник,Доля должников,Отклонение от общего уровня должников
group_in,,,,
Высокие доходы,1535,120,0.078176,-0.010109
Доходы выше среднего,4266,328,0.076887,-0.011398
Доходы ниже среднего,6294,550,0.087385,-0.000900
Средние доходы,7614,742,0.097452,0.009168


Заемщики со средними доходами чаще остальных имеют задолженность по кредитам, а люди с доходами выше среднего выплачивают займы стабильнее

In [53]:
data_client_pivot_purpose_category

,Добросовестный плательщик,Должник,Доля должников,Отклонение от общего уровня должников
purpose_category,,,,
образование,3643,370,0.101565,0.013280
свадьба,2137,186,0.087038,-0.001247
сделка с автомобилем,3903,403,0.103254,0.014969
сделка с недвижимостью,10026,781,0.077897,-0.010387


При выдаче кредита на образование и сделки с автомобилями в 10 случаях из 100 платеж по кредиту будет просрочен, в то время как кредит, выданный на сделку с недвижимостью, более надежный и просрочка платежа возникает только в 8 случаях из 100.